# 4.3 MEV Centrality Patterns

In this notebook, we study **centrality patterns** of MEV-related addresses
in the heterogeneous Ethereum graph.

We focus on addresses that:

- Interact with known DEX routers
- Sit in the neighborhood of routers in the graph
- May act as hubs or bridges in MEV-related subgraphs

We analyze:

- Degree / in-degree / out-degree
- PageRank
- Betweenness centrality (on a sampled subgraph)


## 1. Imports + Load Graph + Transactions

We load:

- The heterogeneous graph `G` from **2.3**
- The cleaned ETH transaction table `tx` (for router annotation)

In [5]:
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from pathlib import Path
import pickle

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["axes.grid"] = True

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
sys.path.append(PROJECT_ROOT)

from src.data.load_data import load_clean_transactions

print("PROJECT_ROOT:", PROJECT_ROOT)


PROJECT_ROOT: /Users/dada/Developer/italy_proj/DataMining/EhereumNetworkAnalysis


In [6]:
# Load graph G
HETERO_GRAPH_PATH = os.path.join(PROJECT_ROOT, "data", "processed", "heterogeneous_graph.gpickle")
with Path(HETERO_GRAPH_PATH).open("rb") as f:
    G = pickle.load(f)

print("Loaded heterogeneous graph G")
print("Nodes:", G.number_of_nodes())
print("Edges:", G.number_of_edges())
print("Directed:", G.is_directed())


Loaded heterogeneous graph G
Nodes: 26447
Edges: 30638
Directed: True


In [7]:
# Load ETH transactions
tx = load_clean_transactions()
print("Transactions:", len(tx))

tx.head()


Transactions: 13268


,hash,from_address,to_address,block_number,value,block_timestamp
0,0xd8ec648861cf4de73f18f9a034623eeded1b26ec7246...,0xa9264494a92ced04747ac84fc9ca5a0b9549b491,0x835033bd90b943fa0d0f8e5382d9dc568d3fbd96,23772289,4.699994e+19,2025-11-11 00:00:11+00:00
1,0x5843a9e865f9b7222ddb376ea2869c50b389c3a0d858...,0xc0ffeebabe5d496b2dde509f9fa189c25cf29671,0xc0ffeebabe5d496b2dde509f9fa189c25cf29671,23772292,5.817089e+19,2025-11-11 00:00:47+00:00
2,0x131571aec26cd23b0134a97341acf9fb0b559b085b68...,0xe50008c1d110da8e56982f46a9188a292ee90a7b,0x1ab4973a48dc892cd9971ece8e01dcc7688f8f23,23772292,3.390013e+18,2025-11-11 00:00:47+00:00
3,0xa1b7caf05dd498111a40ffe269fefb2ae574dde53da0...,0xe40d548eb4fa4d9188fd21723f2fd377456c0876,0x28c6c06298d514db089934071355e5743bf21d60,23772292,7.999922e+18,2025-11-11 00:00:47+00:00
4,0xc1d8e4ffa9e7864d5a38f84aa4532308d411ba35f82e...,0x0eb1665de6473c624dcd087fdeee27418d65ed59,0xa03400e098f4421b34a3a44a1b4e571419517687,23772292,6.318854e+18,2025-11-11 00:00:47+00:00


In [8]:
## 2. Identify Router Contracts in the Graph

#### We reuse the **router address list** from 4.2.

# Example router address list (placeholder — update with your actual dataset)
router_addresses = {
    "uniswap_v2": [
        "0xf164fc0ec4e93095b804a4795bbe1e041497b92a",
    ],
    "uniswap_v3": [
        "0xe592427a0aece92de3edee1f18e0157c05861564",
    ],
    "1inch": [
        "0x1111111254eeb25477b68fb85ed929f73a960582",
    ],
    "0x_exchange": [
        "0xdef1c0ded9bec7f1a1670819833240f027b25eff",
    ],
    "paraswap": [
        "0xdef171fe48cf0115b1d80b88dc8eab59176fee57",
    ],
    "sushi_swap": [
        "0xd9e1ce17f2641f24ae83637ab66a2cca9c378b9f",
    ],
    "balancer": [
        "0xba12222222228d8ba445958a75a07044adaf5ab",
    ],
    "curve": [
        "0x11111112542d85b3ef69ae05771c22d518637fe",
    ],
    "kyber_swap": [
        "0x1c87257f5e8609940bc751a07bb5c9c3e1b0b76",
    ]
}

router_all = set([a.lower() for lst in router_addresses.values() for a in lst])
print("Total unique router addresses:", len(router_all))



Total unique router addresses: 9


### 2.1 Map Routers to Nodes in `G`

We identify which nodes in `G` correspond to DEX router contracts.

In [9]:
# Nodes in G are addresses (lowercase) for EOA / contracts
node_list = pd.Index([str(n).lower() for n in G.nodes()])
router_nodes = node_list[node_list.isin(router_all)]

print("Number of router nodes found in G:", len(router_nodes))
list(router_nodes)[:10]

Number of router nodes found in G: 3


['0x1111111254eeb25477b68fb85ed929f73a960582',
 '0xdef171fe48cf0115b1d80b88dc8eab59176fee57',
 '0xe592427a0aece92de3edee1f18e0157c05861564']

## 3. Build a MEV-Focused Subgraph

We construct a subgraph that contains:

- Router nodes
- Their 1-hop neighbors (addresses interacting with routers)

This gives a MEV-related region of the graph to analyze centrality patterns.

In [10]:
# Collect 1-hop neighbors of routers
mev_node_set = set(router_nodes)

for r in router_nodes:
    # neighbors in directed MultiDiGraph
    if r in G:
        mev_node_set.update(G.predecessors(r))
        mev_node_set.update(G.successors(r))

print("MEV node set size:", len(mev_node_set))

MEV node set size: 12


In [11]:
# Induced subgraph on MEV-related nodes
G_mev = G.subgraph(mev_node_set).copy()

print("G_mev: MEV-focused subgraph")
print("Nodes:", G_mev.number_of_nodes())
print("Edges:", G_mev.number_of_edges())

G_mev: MEV-focused subgraph
Nodes: 12
Edges: 19


## 4. Centrality Measures in the MEV Subgraph

We compute:

- In-degree and out-degree
- Total degree
- PageRank
- (On a sampled subgraph) betweenness centrality

We then inspect the **top nodes** and their `node_type`.

In [12]:
# Degree metrics
in_deg = dict(G_mev.in_degree())
out_deg = dict(G_mev.out_degree())
deg = dict(G_mev.degree())

centrality_df = pd.DataFrame({
    "in_degree": pd.Series(in_deg),
    "out_degree": pd.Series(out_deg),
    "degree": pd.Series(deg),
}).fillna(0).astype(int)

centrality_df.head()

,in_degree,out_degree,degree
0x35c31864370bc6f27e6e700c8244553ed218f46e,0,2,2
0xce6938e9af4ae3d0324952c31fa06f76f94be949,0,2,2
0xdef171fe48cf0115b1d80b88dc8eab59176fee57,1,3,4
0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,4,0,4
0xdac17f958d2ee523a2206206994597c13d831ec7,6,0,6


In [13]:
# Add node_type if available
node_type_map = dict(G_mev.nodes(data="node_type"))
centrality_df["node_type"] = centrality_df.index.map(lambda n: node_type_map.get(n, "Unknown"))

centrality_df["is_router"] = centrality_df.index.str.lower().isin(router_all)

centrality_df.head()

,in_degree,out_degree,degree,node_type,is_router
0x35c31864370bc6f27e6e700c8244553ed218f46e,0,2,2,EOA,False
0xce6938e9af4ae3d0324952c31fa06f76f94be949,0,2,2,EOA,False
0xdef171fe48cf0115b1d80b88dc8eab59176fee57,1,3,4,Contract,True
0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,4,0,4,TokenContract,False
0xdac17f958d2ee523a2206206994597c13d831ec7,6,0,6,TokenContract,False


### 4.1 Top Nodes by Degree

We inspect the top-degree nodes in the MEV subgraph.


In [15]:
top_k = 20

top_degree = (
    centrality_df.sort_values("degree", ascending=False)
                 .head(top_k)
                 .copy()
)

top_degree[["degree", "in_degree", "out_degree", "node_type", "is_router"]]




,degree,in_degree,out_degree,node_type,is_router
0x1111111254eeb25477b68fb85ed929f73a960582,8,5,3,Contract,True
0xdac17f958d2ee523a2206206994597c13d831ec7,6,6,0,TokenContract,False
0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,4,4,0,TokenContract,False
0xdef171fe48cf0115b1d80b88dc8eab59176fee57,4,1,3,Contract,True
0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3,3,0,TokenContract,False
0xe592427a0aece92de3edee1f18e0157c05861564,3,0,3,Contract,True
0x35c31864370bc6f27e6e700c8244553ed218f46e,2,0,2,EOA,False
0xce6938e9af4ae3d0324952c31fa06f76f94be949,2,0,2,EOA,False
0xad9b9d612eba19ea32965748118b2f29d4f6eddb,2,0,2,EOA,False
0x2f28a5e274a99501a3a22888817a4e21507ea2cc,2,0,2,EOA,False


### 4.2 PageRank in the MEV Subgraph

We compute PageRank on `G_mev` to identify nodes that:

- receive flows from other important nodes
- act as "MEV hubs" around routers

In [16]:
print("Computing PageRank on G_mev ...")
pr_mev = nx.pagerank(G_mev, alpha=0.85, max_iter=100)

centrality_df["pagerank"] = pd.Series(pr_mev)

top_pagerank = (
    centrality_df.sort_values("pagerank", ascending=False)
                 .head(top_k)
                 .copy()
)

top_pagerank[["pagerank", "degree", "node_type", "is_router"]]

Computing PageRank on G_mev ...


,pagerank,degree,node_type,is_router
0xdac17f958d2ee523a2206206994597c13d831ec7,0.181381,6,TokenContract,False
0x1111111254eeb25477b68fb85ed929f73a960582,0.157498,8,Contract,True
0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0.143670,4,TokenContract,False
0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0.124815,3,TokenContract,False
0xdef171fe48cf0115b1d80b88dc8eab59176fee57,0.082077,4,Contract,True
0xce6938e9af4ae3d0324952c31fa06f76f94be949,0.044366,2,EOA,False
0x35c31864370bc6f27e6e700c8244553ed218f46e,0.044366,2,EOA,False
0xe20a9ff50b8c3b5baaa17b7f4fdc91169d229989,0.044366,1,EOA,False
0xe592427a0aece92de3edee1f18e0157c05861564,0.044366,3,Contract,True
0x2f28a5e274a99501a3a22888817a4e21507ea2cc,0.044366,2,EOA,False


### 4.3 Betweenness Centrality (Sampled)

Betweenness centrality is expensive on large graphs.

We therefore:

1. Sample a subset of nodes from `G_mev`
2. Convert to a simple undirected graph
3. Compute betweenness centrality on the sampled graph

In [17]:
MAX_BETWEENNESS_NODES = 3000

if G_mev.number_of_nodes() > MAX_BETWEENNESS_NODES:
    sampled_nodes = np.random.choice(list(G_mev.nodes()), size=MAX_BETWEENNESS_NODES, replace=False)
    G_mev_sample = G_mev.subgraph(sampled_nodes).copy()
    print("Using sampled subgraph for betweenness:", G_mev_sample.number_of_nodes(), "nodes")
else:
    G_mev_sample = G_mev
    print("Using full G_mev for betweenness:", G_mev_sample.number_of_nodes(), "nodes")

Using full G_mev for betweenness: 12 nodes


In [18]:
# Convert to undirected simple graph
G_mev_und = nx.Graph(G_mev_sample)

print("Computing betweenness centrality on sampled undirected MEV subgraph ...")
bet_mev = nx.betweenness_centrality(G_mev_und, k=None, normalized=True)

bet_series = pd.Series(bet_mev, name="betweenness")

# Join back
centrality_df["betweenness"] = bet_series

top_betweenness = (
    centrality_df.sort_values("betweenness", ascending=False)
                 .head(top_k)
                 .copy()
)

top_betweenness[["betweenness", "degree", "node_type", "is_router"]]

Computing betweenness centrality on sampled undirected MEV subgraph ...


,betweenness,degree,node_type,is_router
0x1111111254eeb25477b68fb85ed929f73a960582,0.427273,8,Contract,True
0xdac17f958d2ee523a2206206994597c13d831ec7,0.239394,6,TokenContract,False
0xdef171fe48cf0115b1d80b88dc8eab59176fee57,0.200000,4,Contract,True
0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0.103030,4,TokenContract,False
0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0.048485,3,TokenContract,False
0xe592427a0aece92de3edee1f18e0157c05861564,0.018182,3,Contract,True
0x35c31864370bc6f27e6e700c8244553ed218f46e,0.000000,2,EOA,False
0xce6938e9af4ae3d0324952c31fa06f76f94be949,0.000000,2,EOA,False
0xe20a9ff50b8c3b5baaa17b7f4fdc91169d229989,0.000000,1,EOA,False
0x2f28a5e274a99501a3a22888817a4e21507ea2cc,0.000000,2,EOA,False


## 5. Compare Router vs Non-Router Centrality

We summarize centrality statistics for:

- Router nodes
- Non-router nodes in the MEV subgraph

In [19]:
group_stats = (
    centrality_df[["degree", "pagerank", "betweenness", "is_router"]]
    .groupby("is_router")
    .describe()
)

group_stats

degree                                              pagerank  \
           count      mean       std  min  25%  50%  75%  max    count   
is_router                                                                
False        9.0  2.555556  1.589899  1.0  2.0  2.0  3.0  6.0      9.0   
True         3.0  5.000000  2.645751  3.0  3.5  4.0  6.0  8.0      3.0   

                     ...                     betweenness                      \
               mean  ...       75%       max       count      mean       std   
is_router            ...                                                       
False      0.079562  ...  0.124815  0.181381         9.0  0.043434  0.081622   
True       0.094647  ...  0.119787  0.157498         3.0  0.215152  0.204966   

                                                        
                min       25%  50%       75%       max  
is_router                                               
False      0.000000  0.000000  0.0  0.048485  0.239394  
True       0.018182  0.109091  0.2  0.313636  0.427273  

[2 rows x 24 columns]

## 6. Identify Non-Router MEV Candidates

We highlight **non-router** nodes with:

- High degree / pagerank / betweenness
- Located in the router neighborhood

These are strong **MEV candidate addresses** (EOA / other contracts).

In [20]:
candidates = (
    centrality_df[centrality_df["is_router"] == False]
    .sort_values(["pagerank", "degree"], ascending=[False, False])
    .head(30)
    .copy()
)

candidates[["degree", "pagerank", "betweenness", "node_type"]]

,degree,pagerank,betweenness,node_type
0xdac17f958d2ee523a2206206994597c13d831ec7,6,0.181381,0.239394,TokenContract
0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,4,0.143670,0.103030,TokenContract
0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,3,0.124815,0.048485,TokenContract
0x35c31864370bc6f27e6e700c8244553ed218f46e,2,0.044366,0.000000,EOA
0xce6938e9af4ae3d0324952c31fa06f76f94be949,2,0.044366,0.000000,EOA
0x2f28a5e274a99501a3a22888817a4e21507ea2cc,2,0.044366,0.000000,EOA
0xad9b9d612eba19ea32965748118b2f29d4f6eddb,2,0.044366,0.000000,EOA
0xe20a9ff50b8c3b5baaa17b7f4fdc91169d229989,1,0.044366,0.000000,EOA
0xc1b0d5fbaaec5bb6d07b83db890eb576e3393e8a,1,0.044366,0.000000,EOA


# 7. Summary

In this notebook, we:

- Built a **MEV-focused subgraph** around known DEX routers  
- Computed centrality measures (degree, PageRank, betweenness)  
- Compared router vs non-router centrality patterns  
- Identified **non-router MEV candidates** with high centrality
